In [1]:
%matplotlib inline
from selenium import webdriver
from pandas import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import *
import time

In [1]:
path_to_chromedriver = "/Users/GabbyHL/Documents/chromedriver" # Path to access a chrome driver
browser = webdriver.Chrome(executable_path=path_to_chromedriver)

NameError: name 'webdriver' is not defined

In [37]:
url = "https://stats.nba.com/lineups/advanced/"
browser.get(url)

In [137]:
pages = [1,2,3,4,5,6,7,8,9,10,11,12,13]
years = [2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008]

In [178]:
for page in pages:
    year = years[page-1]
    browser.find_element_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[" + str(page) + "]").click()
    time.sleep(15)
    browser.find_element_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[3]/div/div/select/option[1]").click()
    table = browser.find_element_by_class_name('nba-stat-table__overflow')
    lineup_content = []

    for line_id, lines in enumerate(table.text.split('\n')):
        if line_id == 0:
            column_names = lines.split(' ')[1:]
        else:
            lineup_content.append(lines)
        
    player1 = []
    player2 = []
    player3 = []
    player4 = []
    rest = []

    for lineup in lineup_content:
        elements = lineup.split(', ')
        player1.append(elements[0])
        player2.append(elements[1])
        player3.append(elements[2])
        player4.append(elements[3])
        rest.append(elements[4])

    player5 = []
    team = []
    GP = []
    MIN = []
    NETRTG = []
    PACE = []
    rest2 = []

    for row in rest:
        row = row.replace(" Jr.","Jr")
        row = row.replace(" III","III")
        row = row.replace(" II","II")
        row = row.replace(" IV","IV")
        row = row.replace("Mbah a Moute","MbahaMoute")
        row = row.replace("De Colo","DeColo")
        row = row.replace("Yao","M. Yao")
        rest2.append(row)

    for row in rest2:
        elements = row.split(" ")
        player5.append(elements[0] + elements[1])
        team.append(elements[2])
        GP.append(int(elements[3]))
        MIN.append(int(elements[4]))
        NETRTG.append(float(elements[7]))
        PACE.append(float(elements[17]))

    db = pd.DataFrame({"player1": player1,
                      "player2": player2,
                      "player3": player3,
                      "player4": player4,
                      "player5": player5,
                      "team": team,
                      "games_played": GP,
                      "mins_played": MIN,
                      "net_rating": NETRTG,
                      "pace": PACE})

    db['player1'] = db['player1'].str.replace(".","")
    db['player1'] = db['player1'].str.replace(" ","")
    db['player2'] = db['player2'].str.replace(".","")
    db['player2'] = db['player2'].str.replace(" ","")
    db['player3'] = db['player3'].str.replace(".","")
    db['player3'] = db['player3'].str.replace(" ","")
    db['player4'] = db['player4'].str.replace(".","")
    db['player4'] = db['player4'].str.replace(" ","")
    db['player5'] = db['player5'].str.replace(".","")
    db['player5'] = db['player5'].str.replace(" ","")

    db['season'] = year
    db['lineup'] = range(1, len(db) + 1)

    cols = list(db)
    cols.insert(0, cols.pop(cols.index('lineup')))
    db = db.loc[:, cols]
    db
    db.to_csv (r"/Users/GabbyHL/Desktop/NBA-lineupscraping-trial/lineup"+str(year)+".csv", index = False, header=True)

### Team Net Rating

In [3]:
url = "https://stats.nba.com/teams/advanced/"
browser.get(url)

In [4]:
years2 = [2020,2019,2018,2017,2016,2015,2014,2013,2012,2011]
pages2 = [1,2,3,4,5,6,7,8,9,10]

In [43]:
nrdb = pd.DataFrame(columns = ["team","team_nr","season"])
for page in pages2:
    year2 = years2[page-1]
    browser.find_element_by_xpath("/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option["+str(page)+"]").click()
    time.sleep(5)
    table = browser.find_element_by_class_name('nba-stat-table__overflow')
    team_rank = []
    team_name = []
    team_stats = []
    
    for line_id, lines in enumerate(table.text.split('\n')):
        if line_id < 3:
            column_names = lines.split(' ')[1:]
        else:
            if line_id % 3 == 0:
                team_rank.append(lines)
            if line_id % 3 == 1:
                team_name.append(lines)
            if line_id % 3 == 2:
                team_stats.append([float(i) for i in lines.split(' ')])
            
    db = pd.DataFrame({'team': team_name,
                   'team_nr': [i[6] for i in team_stats],
                   'season': year2
                       }
                     )
    nrdb = nrdb.append(db)
    
nrdb

,team,team_nr,season
0,Milwaukee Bucks,10.7,2020
1,Los Angeles Lakers,7.1,2020
2,Toronto Raptors,6.4,2020
3,LA Clippers,6.4,2020
4,Boston Celtics,6.1,2020
5,Denver Nuggets,3.1,2020
6,Miami Heat,3.0,2020
7,Utah Jazz,3.3,2020
8,Dallas Mavericks,5.8,2020
9,Houston Rockets,3.4,2020


In [44]:
nrdb.to_csv (r"/Users/GabbyHL/Desktop/NBA-lineupscraping-trial/teamnr.csv", index = False, header=True)